In [1]:
import gc
import math
import os
import pathlib
import pickle
import random
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm_notebook

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, train_test_split

import lightgbm as lgb
import xgboost as xgb

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
%matplotlib inline

In [3]:
def add_features(
        df,
        input_first_index=None,
        input_last_index=None,
        sample_size=150000,
        holdout_size=50000,
        smootch_windows_size = (3, 5, 7)
    ):
    if input_first_index == None or input_last_index == None:
        input_first_index = df.index.min()
        input_last_index = df.index.max() + 1
        
    
    sample_indexes = random.sample(range(input_first_index, input_last_index), sample_size)
    sample_indexes.sort()
    
    smootch_feature_names = ['smootch_mean_ws_{}'.format(window_size) for window_size in smootch_windows_size]
    acoustic_data_series = df['acoustic_data']
    full_data_indexes = tuple(acoustic_data_series.index.tolist())

    sample_df = df.iloc[sample_indexes]

    sample_df.reset_index(inplace=True)
    sample_df.drop(columns=['index'], inplace=True)

    output_first_index = 0
    output_last_index = len(sample_df) - 1
    
    begin_indexes_set = set()
    end_indexes_set = set()
    
    start_time = time.time()
   
    sample_df_len = sample_df.shape[0]
    for window_size, feature_name in zip(smootch_windows_size, smootch_feature_names):

        feature_values_list = list(range(sample_size))

        half_window_size = window_size // 2

        sample_begin_indexes = sample_indexes[:half_window_size]
        full_data_begin_indexes = set(df.index[sample_begin_indexes].tolist())
        min_full_data_index = min(full_data_indexes)
        
        in_window_full_data_begin_indexes = set(range(input_first_index, input_first_index + half_window_size))              
        in_window_begin_indexes = full_data_begin_indexes.intersection(
            in_window_full_data_begin_indexes
        )
        
        sample_end_indexes = sample_indexes[-half_window_size:]
        full_data_end_indexes = set(df.index[sample_end_indexes].tolist())
        max_full_data_index = max(full_data_end_indexes) + 1
        
        in_window_full_data_end_indexes = set(range(input_last_index - half_window_size, input_last_index))        
        in_window_end_indexes = full_data_end_indexes.intersection(
            in_window_full_data_end_indexes
        )
        if in_window_begin_indexes:
            begin_indexes_set = begin_indexes_set.union(in_window_begin_indexes)
            for i, b_idx in enumerate(sorted(tuple(in_window_begin_indexes))):
                value = sample_df.iloc[i]['acoustic_data']
                temp = acoustic_data_series.iloc[input_first_index:input_first_index + window_size].mean()
                value = value - temp
                feature_values_list[output_first_index + i] = value
                
        if in_window_end_indexes:
            end_indexes_set = end_indexes_set.union(in_window_end_indexes)
            for i, e_idx in enumerate(sorted(tuple(in_window_end_indexes))):
                value = sample_df.iloc[output_last_index - i]['acoustic_data']
                temp = acoustic_data_series.iloc[input_last_index - window_size:].mean()
                value = value - temp
                feature_values_list[output_last_index - i] = value
                
        first_regular_idx = len(begin_indexes_set)
        last_regular_idx = sample_df_len - len(end_indexes_set)
        for i in range(first_regular_idx, last_regular_idx):
            sample_idx = sample_indexes[i]
            feature_values_list[i] = acoustic_data_series.iloc[
                sample_idx - half_window_size:sample_idx + half_window_size
            ].mean()
        sample_df[feature_name] = feature_values_list
        
    holdout_df = None
    if holdout_size > 0:
        holdout_indexes = np.random.randint(0, sample_df.shape[0], holdout_size)
        holdout_df = sample_df.iloc[holdout_indexes]
        holdout_df.reset_index(inplace=True)
        holdout_df.drop(columns=['index'], inplace=True)
        train_indexes = sorted(tuple(set(sample_df.index).difference(set(holdout_indexes))))
        sample_df = sample_df.iloc[train_indexes]
        sample_df.reset_index(inplace=True)
        sample_df.drop(columns=['index'], inplace=True)
    print("Full calculation feature value time (with slicing) {} min:".format((time.time() - start_time) / 60))
    return sample_df, holdout_df

In [4]:
earthquake_margin_indexes =[
    5656573,
    50085877,
    104677355,
    138772452,
    187641819,
    218652629,
    245829584,
    307838916,
    338276286,
    375377847,
    419368879,
    461811622,
    495800224,
    528777114,
    585568143,
    621985672
]

In [5]:
earthquakes_length = [earthquake_margin_indexes[i + 1] - earthquake_margin_indexes[i] for i in range(len(earthquake_margin_indexes) - 1)]

In [6]:
earthquakes_length

[44429304,
 54591478,
 34095097,
 48869367,
 31010810,
 27176955,
 62009332,
 30437370,
 37101561,
 43991032,
 42442743,
 33988602,
 32976890,
 56791029,
 36417529]

In [7]:
complete_earthquakes_length = earthquakes_length[:-1]

In [8]:
#complete_earthquaces_length = complete_earthquaces_length[:-1]

In [9]:
complete_earthquakes_length

[44429304,
 54591478,
 34095097,
 48869367,
 31010810,
 27176955,
 62009332,
 30437370,
 37101561,
 43991032,
 42442743,
 33988602,
 32976890,
 56791029]

In [10]:
params = {
    #'num_leaves': 51,
    'num_leaves': 27,
    #'min_data_in_leaf': 10,
    'min_data_in_leaf': 8,
    'objective':'regression',
    #'max_depth': -1,
    'max_depth': 5,
    'learning_rate': 0.001,
    'boosting': 'gbdt',
    #'feature_fraction': 0.91,
    #'bagging_freq': 1,
    #'bagging_fraction': 0.91,
    #'bagging_seed': 42,
    'metric': 'mae',
    #'lambda_l1': 0.1,
    'verbosity': -1,
    'nthread': 10,
    'random_state': 42
}

In [23]:
def train_models(
        earthquake_margin_indexes,
        complete_earthquakes_length,
        params,
        featured_not_seen_data_df,
        sample_size=None,
        holdout_size=None,
        #not_seen_data_begin=1,
        #not_seen_data_end=5656572 #5656569
    ):
    
    '''
    not_seen_data_df = pd.read_csv(
        '../input/train/train.csv',
        #nrows=100000000,
        names=['acoustic_data', 'time_to_failure'],
        dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32},
        skiprows=not_seen_data_begin,
        nrows=not_seen_data_end
    )
    
    not_seen_data_df, _ = add_features(
        not_seen_data_df,
        sample_size=not_seen_data_df.shape[0],
        holdout_size=0
    )
    '''
    y_not_seen_data = featured_not_seen_data_df['time_to_failure']
    featured_not_seen_data_df = featured_not_seen_data_df[featured_not_seen_data_df.columns.drop('time_to_failure')]
    
    for i in range(len(complete_earthquakes_length)):
        print("i:", i)
        print("complete_earthquakes_length:", complete_earthquakes_length[i])
        print("earthquake_margin_indexes:", earthquake_margin_indexes[i])
        
        earthquake_df = pd.read_csv(
                '../input/train/train.csv',
                #nrows=100000000,
                names=['acoustic_data', 'time_to_failure'],
                dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32},
                skiprows=earthquake_margin_indexes[i],
                nrows=complete_earthquakes_length[i]
            )
        if not sample_size:
            sample_size = complete_earthquakes_length[i] // 10
        if not holdout_size:
            holdout_size = complete_earthquakes_length[i] // 50
        earthquake_add_features_df, holdout_add_features_df = add_features(
                earthquake_df,
                sample_size=sample_size,
                holdout_size=holdout_size
            )
        X_all = earthquake_add_features_df[earthquake_add_features_df.columns.drop('time_to_failure')]
        y_all = earthquake_add_features_df['time_to_failure']

        X_train, X_valid, y_train, y_valid = train_test_split(X_all, y_all, test_size=0.2, random_state=0)

        model = lgb.LGBMRegressor(**params, n_estimators = 20000, n_jobs = 10, num_iterations=40000)
        model.fit(
                X_train,
                y_train,
                eval_set=[(X_train, y_train), (X_valid, y_valid)],
                eval_metric='mae',
                verbose=1000,
                early_stopping_rounds=4000
            )
        X_holdout = holdout_add_features_df[holdout_add_features_df.columns.drop('time_to_failure')]
        y_holdout = holdout_add_features_df['time_to_failure']
        
        y_holdout_predict = model.predict(X_holdout)
        print("earthquake {} mae {}".format(i, mean_absolute_error(y_holdout, y_holdout_predict)))
        
        not_seen_data_predict = model.predict(featured_not_seen_data_df)
        print("not seen data mae: {}".format(mean_absolute_error(y_not_seen_data, not_seen_data_predict)))
        not_seen_data_predict_df = pd.DataFrame({'time_to_failure': not_seen_data_predict})
        not_seen_data_predict_df.to_csv('not_seend_data_earthquake_{}_model_predict.csv'.format(i), index=False)

        #model.save_model('earthquake_{}_model.txt'.format(i))
        #model.Booster(model_file='earthquake_{}_model.txt'.format(i))
        model_file = open('earthquake_{}_model.txt'.format(i), 'wb')
        pickle.dump(model, model_file)
        model_file.flush()
        model_file.close()

    return

In [12]:
%%time
not_seen_data_df = pd.read_csv(
    '../input/train/train.csv',
    #nrows=100000000,
    names=['acoustic_data', 'time_to_failure'],
    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32},
    skiprows=1,
    nrows=5656572
)

CPU times: user 1.42 s, sys: 88 ms, total: 1.51 s
Wall time: 1.51 s


In [13]:
%%time
featured_not_seen_data_df, _ = add_features(
    not_seen_data_df,
    sample_size=not_seen_data_df.shape[0] // 20,
    holdout_size=0
)

Full calculation feature value time (with slicing) 3.067034912109375 min:
CPU times: user 3min 5s, sys: 244 ms, total: 3min 5s
Wall time: 3min 5s


In [21]:
train_models(earthquake_margin_indexes, complete_earthquakes_length, params, featured_not_seen_data_df)

i: 0
complete_earthquakes_length: 44429304
earthquake_margin_indexes: 5656573
i: 1
complete_earthquakes_length: 54591478
earthquake_margin_indexes: 50085877
i: 2
complete_earthquakes_length: 34095097
earthquake_margin_indexes: 104677355
i: 3
complete_earthquakes_length: 48869367
earthquake_margin_indexes: 138772452
i: 4
complete_earthquakes_length: 31010810
earthquake_margin_indexes: 187641819
i: 5
complete_earthquakes_length: 27176955
earthquake_margin_indexes: 218652629
i: 6
complete_earthquakes_length: 62009332
earthquake_margin_indexes: 245829584
i: 7
complete_earthquakes_length: 30437370
earthquake_margin_indexes: 307838916
i: 8
complete_earthquakes_length: 37101561
earthquake_margin_indexes: 338276286
i: 9
complete_earthquakes_length: 43991032
earthquake_margin_indexes: 375377847
i: 10
complete_earthquakes_length: 42442743
earthquake_margin_indexes: 419368879
i: 11
complete_earthquakes_length: 33988602
earthquake_margin_indexes: 461811622
i: 12
complete_earthquakes_length: 329768

In [ ]:
%%time
train_models(earthquake_margin_indexes[:5], complete_earthquakes_length, params, featured_not_seen_data_df)

i: 0
complete_earthquakes_length: 44429304
earthquake_margin_indexes: 5656573


In [ ]:
#featured_not_seen_data_df.drop?

In [17]:
print(len(complete_earthquakes_length))
print(len(earthquake_margin_indexes))

14
16


In [ ]:
lgb.__version__

In [ ]:
Training until validation scores don't improve for 4000 rounds.
[1000]	training's l1: 2.85332	valid_1's l1: 2.85467
[2000]	training's l1: 2.84694	valid_1's l1: 2.84851
[3000]	training's l1: 2.84517	valid_1's l1: 2.84683
[4000]	training's l1: 2.84444	valid_1's l1: 2.84614
[5000]	training's l1: 2.84409	valid_1's l1: 2.84583
[6000]	training's l1: 2.84391	valid_1's l1: 2.84568
[7000]	training's l1: 2.8438	valid_1's l1: 2.84561
[8000]	training's l1: 2.84371	valid_1's l1: 2.84555
[9000]	training's l1: 2.84364	valid_1's l1: 2.84551
[10000]	training's l1: 2.84357	valid_1's l1: 2.84547
[11000]	training's l1: 2.84351	valid_1's l1: 2.84544
[12000]	training's l1: 2.84345	valid_1's l1: 2.84541
[13000]	training's l1: 2.84339	valid_1's l1: 2.84539
[14000]	training's l1: 2.84334	valid_1's l1: 2.84536
[15000]	training's l1: 2.84329	valid_1's l1: 2.84534
[16000]	training's l1: 2.84323	valid_1's l1: 2.84532
[17000]	training's l1: 2.84319	valid_1's l1: 2.84531
[18000]	training's l1: 2.84314	valid_1's l1: 2.84529
[19000]	training's l1: 2.8431	valid_1's l1: 2.84528
[20000]	training's l1: 2.84306	valid_1's l1: 2.84527
[21000]	training's l1: 2.84302	valid_1's l1: 2.84526
[22000]	training's l1: 2.84298	valid_1's l1: 2.84525
[23000]	training's l1: 2.84294	valid_1's l1: 2.84524
[24000]	training's l1: 2.84291	valid_1's l1: 2.84524
[25000]	training's l1: 2.84287	valid_1's l1: 2.84523
[26000]	training's l1: 2.84284	valid_1's l1: 2.84523
[27000]	training's l1: 2.8428	valid_1's l1: 2.84523
[28000]	training's l1: 2.84277	valid_1's l1: 2.84523
[29000]	training's l1: 2.84274	valid_1's l1: 2.84523
[30000]	training's l1: 2.84271	valid_1's l1: 2.84522
[31000]	training's l1: 2.84267	valid_1's l1: 2.84522
[32000]	training's l1: 2.84264	valid_1's l1: 2.84521
[33000]	training's l1: 2.84261	valid_1's l1: 2.84521
[34000]	training's l1: 2.84258	valid_1's l1: 2.84521
[35000]	training's l1: 2.84256	valid_1's l1: 2.8452
[36000]	training's l1: 2.84253	valid_1's l1: 2.8452
[37000]	training's l1: 2.8425	valid_1's l1: 2.8452
[38000]	training's l1: 2.84247	valid_1's l1: 2.84519
[39000]	training's l1: 2.84244	valid_1's l1: 2.84519
[40000]	training's l1: 2.84241	valid_1's l1: 2.84519
Did not meet early stopping. Best iteration is:
[40000]	training's l1: 2.84241	valid_1's l1: 2.84519
earthquake 0 mae 2.8432238740079154
not seen data mae: 4.975486288323955

In [ ]:
Full calculation feature value time (with slicing) 49.01300070285797 min:

/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

Training until validation scores don't improve for 4000 rounds.
[1000]	training's l1: 2.85355	valid_1's l1: 2.84966
[2000]	training's l1: 2.84706	valid_1's l1: 2.84321
[3000]	training's l1: 2.84528	valid_1's l1: 2.84152
[4000]	training's l1: 2.84457	valid_1's l1: 2.84088
[5000]	training's l1: 2.84421	valid_1's l1: 2.84059
[6000]	training's l1: 2.84399	valid_1's l1: 2.84044
[7000]	training's l1: 2.84386	valid_1's l1: 2.84038
[8000]	training's l1: 2.84375	valid_1's l1: 2.84034
[9000]	training's l1: 2.84365	valid_1's l1: 2.8403
[10000]	training's l1: 2.84356	valid_1's l1: 2.84026
[11000]	training's l1: 2.84348	valid_1's l1: 2.84024
[12000]	training's l1: 2.84339	valid_1's l1: 2.84022
[13000]	training's l1: 2.84331	valid_1's l1: 2.84021
[14000]	training's l1: 2.84323	valid_1's l1: 2.8402
[15000]	training's l1: 2.84316	valid_1's l1: 2.84019
[16000]	training's l1: 2.84309	valid_1's l1: 2.84018
[17000]	training's l1: 2.84302	valid_1's l1: 2.84017
[18000]	training's l1: 2.84294	valid_1's l1: 2.84017
[19000]	training's l1: 2.84288	valid_1's l1: 2.84017
[20000]	training's l1: 2.84281	valid_1's l1: 2.84016
[21000]	training's l1: 2.84274	valid_1's l1: 2.84016
[22000]	training's l1: 2.84267	valid_1's l1: 2.84015
[23000]	training's l1: 2.84261	valid_1's l1: 2.84015
[24000]	training's l1: 2.84255	valid_1's l1: 2.84015
[25000]	training's l1: 2.8425	valid_1's l1: 2.84015
[26000]	training's l1: 2.84244	valid_1's l1: 2.84015
[27000]	training's l1: 2.84238	valid_1's l1: 2.84015
[28000]	training's l1: 2.84232	valid_1's l1: 2.84016
Early stopping, best iteration is:
[24063]	training's l1: 2.84255	valid_1's l1: 2.84015
earthquake 0 mae 2.8414997116203136
not seen data mae: 4.974842378258406
Full calculation feature value time (with slicing) 48.64994461933772 min:

/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

Training until validation scores don't improve for 4000 rounds.
[1000]	training's l1: 3.49478	valid_1's l1: 3.49067
[2000]	training's l1: 3.48463	valid_1's l1: 3.48035
[3000]	training's l1: 3.4815	valid_1's l1: 3.47725
[4000]	training's l1: 3.48016	valid_1's l1: 3.47597
[5000]	training's l1: 3.4795	valid_1's l1: 3.47542
[6000]	training's l1: 3.47915	valid_1's l1: 3.47517
[7000]	training's l1: 3.47894	valid_1's l1: 3.47504
[8000]	training's l1: 3.47878	valid_1's l1: 3.47496
[9000]	training's l1: 3.47865	valid_1's l1: 3.47489
[10000]	training's l1: 3.47853	valid_1's l1: 3.47484
[11000]	training's l1: 3.47841	valid_1's l1: 3.4748
[12000]	training's l1: 3.47829	valid_1's l1: 3.47476
[13000]	training's l1: 3.47819	valid_1's l1: 3.47472
[14000]	training's l1: 3.47808	valid_1's l1: 3.47469
[15000]	training's l1: 3.47798	valid_1's l1: 3.47466
[16000]	training's l1: 3.47789	valid_1's l1: 3.47463
[17000]	training's l1: 3.4778	valid_1's l1: 3.47461
[18000]	training's l1: 3.4777	valid_1's l1: 3.47459
[19000]	training's l1: 3.47762	valid_1's l1: 3.47457
[20000]	training's l1: 3.47753	valid_1's l1: 3.47456
[21000]	training's l1: 3.47744	valid_1's l1: 3.47456
[22000]	training's l1: 3.47737	valid_1's l1: 3.47456
[23000]	training's l1: 3.47729	valid_1's l1: 3.47456
[24000]	training's l1: 3.47721	valid_1's l1: 3.47455
[25000]	training's l1: 3.47714	valid_1's l1: 3.47455
[26000]	training's l1: 3.47706	valid_1's l1: 3.47455
[27000]	training's l1: 3.47699	valid_1's l1: 3.47454
[28000]	training's l1: 3.47692	valid_1's l1: 3.47453
[29000]	training's l1: 3.47684	valid_1's l1: 3.47453
[30000]	training's l1: 3.47677	valid_1's l1: 3.47452
[31000]	training's l1: 3.4767	valid_1's l1: 3.47452
[32000]	training's l1: 3.47663	valid_1's l1: 3.47451
[33000]	training's l1: 3.47657	valid_1's l1: 3.47451
[34000]	training's l1: 3.47651	valid_1's l1: 3.47451
[35000]	training's l1: 3.47644	valid_1's l1: 3.47451
[36000]	training's l1: 3.47637	valid_1's l1: 3.4745
[37000]	training's l1: 3.47631	valid_1's l1: 3.4745
[38000]	training's l1: 3.47624	valid_1's l1: 3.47449
[39000]	training's l1: 3.47617	valid_1's l1: 3.4745
[40000]	training's l1: 3.47611	valid_1's l1: 3.4745
Did not meet early stopping. Best iteration is:
[40000]	training's l1: 3.47611	valid_1's l1: 3.4745
earthquake 1 mae 3.4807936077768415
not seen data mae: 6.327089362416579
Full calculation feature value time (with slicing) 48.33311646382014 min:

/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

Training until validation scores don't improve for 4000 rounds.
[1000]	training's l1: 2.17379	valid_1's l1: 2.17171
[2000]	training's l1: 2.16407	valid_1's l1: 2.1618
[3000]	training's l1: 2.16099	valid_1's l1: 2.1587
[4000]	training's l1: 2.15975	valid_1's l1: 2.15751
[5000]	training's l1: 2.15915	valid_1's l1: 2.15696
[6000]	training's l1: 2.15887	valid_1's l1: 2.15673
[7000]	training's l1: 2.15871	valid_1's l1: 2.15662
[8000]	training's l1: 2.1586	valid_1's l1: 2.15655
[9000]	training's l1: 2.1585	valid_1's l1: 2.15649
[10000]	training's l1: 2.1584	valid_1's l1: 2.15644
[11000]	training's l1: 2.15831	valid_1's l1: 2.1564
[12000]	training's l1: 2.15824	valid_1's l1: 2.15637
[13000]	training's l1: 2.15817	valid_1's l1: 2.15634
[14000]	training's l1: 2.1581	valid_1's l1: 2.15631
[15000]	training's l1: 2.15803	valid_1's l1: 2.15628
[16000]	training's l1: 2.15797	valid_1's l1: 2.15627
[17000]	training's l1: 2.15791	valid_1's l1: 2.15626
[18000]	training's l1: 2.15785	valid_1's l1: 2.15625
[19000]	training's l1: 2.1578	valid_1's l1: 2.15624
[20000]	training's l1: 2.15775	valid_1's l1: 2.15624
[21000]	training's l1: 2.15769	valid_1's l1: 2.15623
[22000]	training's l1: 2.15764	valid_1's l1: 2.15622
[23000]	training's l1: 2.1576	valid_1's l1: 2.15622
[24000]	training's l1: 2.15755	valid_1's l1: 2.15621
[25000]	training's l1: 2.1575	valid_1's l1: 2.15621
[26000]	training's l1: 2.15746	valid_1's l1: 2.1562
[27000]	training's l1: 2.15741	valid_1's l1: 2.1562
[28000]	training's l1: 2.15736	valid_1's l1: 2.15619
[29000]	training's l1: 2.15732	valid_1's l1: 2.15619
[30000]	training's l1: 2.15728	valid_1's l1: 2.15619
[31000]	training's l1: 2.15723	valid_1's l1: 2.15618
[32000]	training's l1: 2.15719	valid_1's l1: 2.15618
[33000]	training's l1: 2.15714	valid_1's l1: 2.15617
[34000]	training's l1: 2.15711	valid_1's l1: 2.15617
[35000]	training's l1: 2.15707	valid_1's l1: 2.15618
[36000]	training's l1: 2.15703	valid_1's l1: 2.15618
[37000]	training's l1: 2.15699	valid_1's l1: 2.15618
Early stopping, best iteration is:
[33808]	training's l1: 2.15711	valid_1's l1: 2.15617
earthquake 2 mae 2.1605117591836964
not seen data mae: 3.632222302971268
Full calculation feature value time (with slicing) 48.214682531356814 min:

/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

Training until validation scores don't improve for 4000 rounds.
[1000]	training's l1: 3.11891	valid_1's l1: 3.11993
[2000]	training's l1: 3.10623	valid_1's l1: 3.10769
[3000]	training's l1: 3.10228	valid_1's l1: 3.10393
[4000]	training's l1: 3.10066	valid_1's l1: 3.10239
[5000]	training's l1: 3.09987	valid_1's l1: 3.10169
[6000]	training's l1: 3.09948	valid_1's l1: 3.10137
[7000]	training's l1: 3.09926	valid_1's l1: 3.1012
[8000]	training's l1: 3.09906	valid_1's l1: 3.10106
[9000]	training's l1: 3.0989	valid_1's l1: 3.10095
[10000]	training's l1: 3.09876	valid_1's l1: 3.10086
[11000]	training's l1: 3.09862	valid_1's l1: 3.10079
[12000]	training's l1: 3.0985	valid_1's l1: 3.10073
[13000]	training's l1: 3.09839	valid_1's l1: 3.10069
[14000]	training's l1: 3.09828	valid_1's l1: 3.10065
[15000]	training's l1: 3.09818	valid_1's l1: 3.10062
[16000]	training's l1: 3.09809	valid_1's l1: 3.10059
[17000]	training's l1: 3.09799	valid_1's l1: 3.10056
[18000]	training's l1: 3.09791	valid_1's l1: 3.10053
[19000]	training's l1: 3.09782	valid_1's l1: 3.10051
[20000]	training's l1: 3.09774	valid_1's l1: 3.10049
[21000]	training's l1: 3.09766	valid_1's l1: 3.10048
[22000]	training's l1: 3.09758	valid_1's l1: 3.10046
[23000]	training's l1: 3.0975	valid_1's l1: 3.10045
[24000]	training's l1: 3.09743	valid_1's l1: 3.10044
[25000]	training's l1: 3.09736	valid_1's l1: 3.10043
[26000]	training's l1: 3.09728	valid_1's l1: 3.10042
[27000]	training's l1: 3.09721	valid_1's l1: 3.10041
[28000]	training's l1: 3.09714	valid_1's l1: 3.1004
[29000]	training's l1: 3.09708	valid_1's l1: 3.10039
[30000]	training's l1: 3.09701	valid_1's l1: 3.10039
[31000]	training's l1: 3.09694	valid_1's l1: 3.10038
[32000]	training's l1: 3.09687	valid_1's l1: 3.10038
[33000]	training's l1: 3.0968	valid_1's l1: 3.10037
[34000]	training's l1: 3.09674	valid_1's l1: 3.10036
[35000]	training's l1: 3.09668	valid_1's l1: 3.10036
[36000]	training's l1: 3.09662	valid_1's l1: 3.10036
[37000]	training's l1: 3.09656	valid_1's l1: 3.10035
[38000]	training's l1: 3.09651	valid_1's l1: 3.10035
[39000]	training's l1: 3.09645	valid_1's l1: 3.10034
[40000]	training's l1: 3.09639	valid_1's l1: 3.10034
Did not meet early stopping. Best iteration is:
[40000]	training's l1: 3.09639	valid_1's l1: 3.10034
earthquake 3 mae 3.104666028125392
not seen data mae: 5.555005870175777
Full calculation feature value time (with slicing) 48.83295087019602 min:

/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

Training until validation scores don't improve for 4000 rounds.
[1000]	training's l1: 1.98315	valid_1's l1: 1.98359
[2000]	training's l1: 1.97597	valid_1's l1: 1.97652
[3000]	training's l1: 1.97373	valid_1's l1: 1.97438
[4000]	training's l1: 1.97285	valid_1's l1: 1.9736
[5000]	training's l1: 1.97244	valid_1's l1: 1.97327
[6000]	training's l1: 1.97223	valid_1's l1: 1.97313
[7000]	training's l1: 1.9721	valid_1's l1: 1.97306
[8000]	training's l1: 1.97201	valid_1's l1: 1.97301
[9000]	training's l1: 1.97193	valid_1's l1: 1.97299
[10000]	training's l1: 1.97185	valid_1's l1: 1.97296
[11000]	training's l1: 1.97179	valid_1's l1: 1.97294
[12000]	training's l1: 1.97173	valid_1's l1: 1.97293
[13000]	training's l1: 1.97167	valid_1's l1: 1.97292
[14000]	training's l1: 1.97161	valid_1's l1: 1.9729
[15000]	training's l1: 1.97156	valid_1's l1: 1.97289
[16000]	training's l1: 1.97151	valid_1's l1: 1.97288
[17000]	training's l1: 1.97145	valid_1's l1: 1.97287
[18000]	training's l1: 1.9714	valid_1's l1: 1.97287
[19000]	training's l1: 1.97135	valid_1's l1: 1.97286
[20000]	training's l1: 1.97131	valid_1's l1: 1.97286
[21000]	training's l1: 1.97126	valid_1's l1: 1.97285
[22000]	training's l1: 1.97122	valid_1's l1: 1.97285
[23000]	training's l1: 1.97117	valid_1's l1: 1.97285
[24000]	training's l1: 1.97113	valid_1's l1: 1.97284
[25000]	training's l1: 1.97109	valid_1's l1: 1.97284
[26000]	training's l1: 1.97105	valid_1's l1: 1.97284
[27000]	training's l1: 1.97101	valid_1's l1: 1.97284
[28000]	training's l1: 1.97097	valid_1's l1: 1.97283
[29000]	training's l1: 1.97092	valid_1's l1: 1.97283
[30000]	training's l1: 1.97088	valid_1's l1: 1.97282
[31000]	training's l1: 1.97084	valid_1's l1: 1.97282
[32000]	training's l1: 1.9708	valid_1's l1: 1.97282
[33000]	training's l1: 1.97076	valid_1's l1: 1.97283
[34000]	training's l1: 1.97072	valid_1's l1: 1.97283
Early stopping, best iteration is:
[30596]	training's l1: 1.97085	valid_1's l1: 1.97282
earthquake 4 mae 1.9692684614550362
not seen data mae: 3.221705830518252
Full calculation feature value time (with slicing) 49.619407598177595 min:

/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

Training until validation scores don't improve for 4000 rounds.
[1000]	training's l1: 1.74245	valid_1's l1: 1.74157
[2000]	training's l1: 1.73641	valid_1's l1: 1.73575
[3000]	training's l1: 1.73444	valid_1's l1: 1.7339
[4000]	training's l1: 1.73363	valid_1's l1: 1.73317
[5000]	training's l1: 1.73325	valid_1's l1: 1.73285
[6000]	training's l1: 1.73305	valid_1's l1: 1.73271
[7000]	training's l1: 1.73293	valid_1's l1: 1.73263
[8000]	training's l1: 1.73283	valid_1's l1: 1.73258
[9000]	training's l1: 1.73275	valid_1's l1: 1.73254
[10000]	training's l1: 1.73268	valid_1's l1: 1.73251
[11000]	training's l1: 1.73261	valid_1's l1: 1.73248
[12000]	training's l1: 1.73255	valid_1's l1: 1.73246
[13000]	training's l1: 1.73248	valid_1's l1: 1.73244
[14000]	training's l1: 1.73243	valid_1's l1: 1.73242
[15000]	training's l1: 1.73238	valid_1's l1: 1.73241
[16000]	training's l1: 1.73233	valid_1's l1: 1.73239
[17000]	training's l1: 1.73227	valid_1's l1: 1.73238
[18000]	training's l1: 1.73223	valid_1's l1: 1.73237
[19000]	training's l1: 1.73219	valid_1's l1: 1.73236
[20000]	training's l1: 1.73215	valid_1's l1: 1.73236
[21000]	training's l1: 1.73211	valid_1's l1: 1.73236
[22000]	training's l1: 1.73207	valid_1's l1: 1.73235
[23000]	training's l1: 1.73203	valid_1's l1: 1.73235
[24000]	training's l1: 1.732	valid_1's l1: 1.73234
[25000]	training's l1: 1.73196	valid_1's l1: 1.73234
[26000]	training's l1: 1.73192	valid_1's l1: 1.73234
[27000]	training's l1: 1.73188	valid_1's l1: 1.73233
[28000]	training's l1: 1.73184	valid_1's l1: 1.73233
[29000]	training's l1: 1.7318	valid_1's l1: 1.73232
[30000]	training's l1: 1.73176	valid_1's l1: 1.73232
[31000]	training's l1: 1.73172	valid_1's l1: 1.73232
[32000]	training's l1: 1.73169	valid_1's l1: 1.73232
[33000]	training's l1: 1.73165	valid_1's l1: 1.73232
[34000]	training's l1: 1.73161	valid_1's l1: 1.73232
[35000]	training's l1: 1.73158	valid_1's l1: 1.73232
[36000]	training's l1: 1.73155	valid_1's l1: 1.73231
[37000]	training's l1: 1.73152	valid_1's l1: 1.73231
[38000]	training's l1: 1.73149	valid_1's l1: 1.73231
[39000]	training's l1: 1.73145	valid_1's l1: 1.73231
[40000]	training's l1: 1.73141	valid_1's l1: 1.73231
Did not meet early stopping. Best iteration is:
[40000]	training's l1: 1.73141	valid_1's l1: 1.73231
earthquake 5 mae 1.7320401261840532
not seen data mae: 2.762850373105831
Full calculation feature value time (with slicing) 49.52320881287257 min:

/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

Training until validation scores don't improve for 4000 rounds.
[1000]	training's l1: 3.98032	valid_1's l1: 3.97625
[2000]	training's l1: 3.97022	valid_1's l1: 3.96631
[3000]	training's l1: 3.96714	valid_1's l1: 3.96336
[4000]	training's l1: 3.96589	valid_1's l1: 3.9622
[5000]	training's l1: 3.96534	valid_1's l1: 3.96176
[6000]	training's l1: 3.96505	valid_1's l1: 3.96157
[7000]	training's l1: 3.96485	valid_1's l1: 3.96147
[8000]	training's l1: 3.96469	valid_1's l1: 3.96139
[9000]	training's l1: 3.96453	valid_1's l1: 3.96132
[10000]	training's l1: 3.9644	valid_1's l1: 3.96127
[11000]	training's l1: 3.96428	valid_1's l1: 3.96123
[12000]	training's l1: 3.96416	valid_1's l1: 3.9612
[13000]	training's l1: 3.96405	valid_1's l1: 3.96118
[14000]	training's l1: 3.96395	valid_1's l1: 3.96115
[15000]	training's l1: 3.96386	valid_1's l1: 3.96113
[16000]	training's l1: 3.96376	valid_1's l1: 3.96112
[17000]	training's l1: 3.96365	valid_1's l1: 3.9611
[18000]	training's l1: 3.96355	valid_1's l1: 3.96108
[19000]	training's l1: 3.96346	valid_1's l1: 3.96107
[20000]	training's l1: 3.96337	valid_1's l1: 3.96105
[21000]	training's l1: 3.96327	valid_1's l1: 3.96104
[22000]	training's l1: 3.96319	valid_1's l1: 3.96104
[23000]	training's l1: 3.96309	valid_1's l1: 3.96103
[24000]	training's l1: 3.96301	valid_1's l1: 3.96103
[25000]	training's l1: 3.96293	valid_1's l1: 3.96103
[26000]	training's l1: 3.96285	valid_1's l1: 3.96102
[27000]	training's l1: 3.96277	valid_1's l1: 3.96101
[28000]	training's l1: 3.9627	valid_1's l1: 3.96101
[29000]	training's l1: 3.96262	valid_1's l1: 3.96101
[30000]	training's l1: 3.96256	valid_1's l1: 3.96101
[31000]	training's l1: 3.96249	valid_1's l1: 3.96101
[32000]	training's l1: 3.96242	valid_1's l1: 3.961
[33000]	training's l1: 3.96234	valid_1's l1: 3.961
[34000]	training's l1: 3.96227	valid_1's l1: 3.961
[35000]	training's l1: 3.9622	valid_1's l1: 3.96101
[36000]	training's l1: 3.96212	valid_1's l1: 3.96101
Early stopping, best iteration is:
[32580]	training's l1: 3.96237	valid_1's l1: 3.961
earthquake 6 mae 3.9652793726011524
not seen data mae: 7.260232979477146
Full calculation feature value time (with slicing) 48.22393831411998 min:

/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

Training until validation scores don't improve for 4000 rounds.
[1000]	training's l1: 1.94562	valid_1's l1: 1.94444
[2000]	training's l1: 1.93796	valid_1's l1: 1.93694
[3000]	training's l1: 1.93554	valid_1's l1: 1.9346
[4000]	training's l1: 1.93457	valid_1's l1: 1.93369
[5000]	training's l1: 1.93412	valid_1's l1: 1.93328
[6000]	training's l1: 1.93391	valid_1's l1: 1.93312
[7000]	training's l1: 1.93378	valid_1's l1: 1.93303
[8000]	training's l1: 1.93367	valid_1's l1: 1.93296
[9000]	training's l1: 1.93358	valid_1's l1: 1.93291
[10000]	training's l1: 1.93349	valid_1's l1: 1.93287
[11000]	training's l1: 1.93342	valid_1's l1: 1.93283
[12000]	training's l1: 1.93335	valid_1's l1: 1.93281
[13000]	training's l1: 1.93328	valid_1's l1: 1.93278
[14000]	training's l1: 1.93322	valid_1's l1: 1.93276
[15000]	training's l1: 1.93316	valid_1's l1: 1.93274
[16000]	training's l1: 1.9331	valid_1's l1: 1.93273
[17000]	training's l1: 1.93304	valid_1's l1: 1.93271
[18000]	training's l1: 1.93299	valid_1's l1: 1.9327
[19000]	training's l1: 1.93294	valid_1's l1: 1.93269
[20000]	training's l1: 1.93289	valid_1's l1: 1.93268
[21000]	training's l1: 1.93284	valid_1's l1: 1.93268
[22000]	training's l1: 1.9328	valid_1's l1: 1.93268
[23000]	training's l1: 1.93276	valid_1's l1: 1.93267
[24000]	training's l1: 1.93272	valid_1's l1: 1.93267
[25000]	training's l1: 1.93268	valid_1's l1: 1.93266
[26000]	training's l1: 1.93263	valid_1's l1: 1.93265
[27000]	training's l1: 1.93259	valid_1's l1: 1.93265
[28000]	training's l1: 1.93255	valid_1's l1: 1.93264
[29000]	training's l1: 1.93251	valid_1's l1: 1.93264
[30000]	training's l1: 1.93247	valid_1's l1: 1.93264
[31000]	training's l1: 1.93244	valid_1's l1: 1.93263
[32000]	training's l1: 1.9324	valid_1's l1: 1.93263
[33000]	training's l1: 1.93236	valid_1's l1: 1.93263
[34000]	training's l1: 1.93232	valid_1's l1: 1.93263
[35000]	training's l1: 1.93229	valid_1's l1: 1.93263
[36000]	training's l1: 1.93225	valid_1's l1: 1.93263
[37000]	training's l1: 1.93222	valid_1's l1: 1.93263
[38000]	training's l1: 1.93219	valid_1's l1: 1.93263
[39000]	training's l1: 1.93216	valid_1's l1: 1.93263
Early stopping, best iteration is:
[35983]	training's l1: 1.93225	valid_1's l1: 1.93263
earthquake 7 mae 1.933108089053609
not seen data mae: 3.1379145540556395
Full calculation feature value time (with slicing) 49.213378763198854 min:

/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

Training until validation scores don't improve for 4000 rounds.
[1000]	training's l1: 2.37458	valid_1's l1: 2.37148
[2000]	training's l1: 2.36594	valid_1's l1: 2.36283
[3000]	training's l1: 2.36324	valid_1's l1: 2.36014
[4000]	training's l1: 2.36217	valid_1's l1: 2.35914
[5000]	training's l1: 2.36166	valid_1's l1: 2.3587
[6000]	training's l1: 2.36143	valid_1's l1: 2.35854
[7000]	training's l1: 2.36128	valid_1's l1: 2.35845
[8000]	training's l1: 2.36116	valid_1's l1: 2.35839
[9000]	training's l1: 2.36105	valid_1's l1: 2.35835
[10000]	training's l1: 2.36096	valid_1's l1: 2.35831
[11000]	training's l1: 2.36088	valid_1's l1: 2.35828
[12000]	training's l1: 2.3608	valid_1's l1: 2.35826
[13000]	training's l1: 2.36072	valid_1's l1: 2.35824
[14000]	training's l1: 2.36065	valid_1's l1: 2.35821
[15000]	training's l1: 2.36058	valid_1's l1: 2.35819
[16000]	training's l1: 2.36051	valid_1's l1: 2.35817
[17000]	training's l1: 2.36045	valid_1's l1: 2.35816
[18000]	training's l1: 2.36039	valid_1's l1: 2.35815
[19000]	training's l1: 2.36033	valid_1's l1: 2.35814
[20000]	training's l1: 2.36027	valid_1's l1: 2.35813
[21000]	training's l1: 2.36021	valid_1's l1: 2.35813
[22000]	training's l1: 2.36016	valid_1's l1: 2.35812
[23000]	training's l1: 2.3601	valid_1's l1: 2.35811
[24000]	training's l1: 2.36005	valid_1's l1: 2.35811
[25000]	training's l1: 2.35999	valid_1's l1: 2.3581
[26000]	training's l1: 2.35993	valid_1's l1: 2.3581
[27000]	training's l1: 2.35988	valid_1's l1: 2.35809
[28000]	training's l1: 2.35983	valid_1's l1: 2.35809
[29000]	training's l1: 2.35978	valid_1's l1: 2.35809
[30000]	training's l1: 2.35972	valid_1's l1: 2.35809
[31000]	training's l1: 2.35968	valid_1's l1: 2.35808
[32000]	training's l1: 2.35963	valid_1's l1: 2.35808
[33000]	training's l1: 2.35958	valid_1's l1: 2.35808
[34000]	training's l1: 2.35954	valid_1's l1: 2.35808
[35000]	training's l1: 2.3595	valid_1's l1: 2.35808
[36000]	training's l1: 2.35945	valid_1's l1: 2.35809
[37000]	training's l1: 2.35941	valid_1's l1: 2.35809
Early stopping, best iteration is:
[33089]	training's l1: 2.35958	valid_1's l1: 2.35808
earthquake 8 mae 2.3604702546694556
not seen data mae: 4.024417336485413
Full calculation feature value time (with slicing) 49.25556689898173 min:

/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

Training until validation scores don't improve for 4000 rounds.
[1000]	training's l1: 2.81774	valid_1's l1: 2.81345
[2000]	training's l1: 2.80817	valid_1's l1: 2.80398
[3000]	training's l1: 2.80522	valid_1's l1: 2.80111
[4000]	training's l1: 2.80403	valid_1's l1: 2.80002
[5000]	training's l1: 2.8035	valid_1's l1: 2.79956
[6000]	training's l1: 2.80326	valid_1's l1: 2.79941
[7000]	training's l1: 2.80311	valid_1's l1: 2.79932
[8000]	training's l1: 2.80298	valid_1's l1: 2.79925
[9000]	training's l1: 2.80286	valid_1's l1: 2.79919
[10000]	training's l1: 2.80274	valid_1's l1: 2.79914
[11000]	training's l1: 2.80263	valid_1's l1: 2.7991
[12000]	training's l1: 2.80254	valid_1's l1: 2.79907
[13000]	training's l1: 2.80245	valid_1's l1: 2.79905
[14000]	training's l1: 2.80237	valid_1's l1: 2.79903
[15000]	training's l1: 2.80229	valid_1's l1: 2.79901
[16000]	training's l1: 2.80221	valid_1's l1: 2.799
[17000]	training's l1: 2.80213	valid_1's l1: 2.79898
[18000]	training's l1: 2.80205	valid_1's l1: 2.79897
[19000]	training's l1: 2.80198	valid_1's l1: 2.79896
[20000]	training's l1: 2.80191	valid_1's l1: 2.79896
[21000]	training's l1: 2.80184	valid_1's l1: 2.79895
[22000]	training's l1: 2.80178	valid_1's l1: 2.79894
[23000]	training's l1: 2.80172	valid_1's l1: 2.79894
[24000]	training's l1: 2.80166	valid_1's l1: 2.79893
[25000]	training's l1: 2.80159	valid_1's l1: 2.79893
[26000]	training's l1: 2.80153	valid_1's l1: 2.79892
[27000]	training's l1: 2.80147	valid_1's l1: 2.79892
[28000]	training's l1: 2.80141	valid_1's l1: 2.79891
[29000]	training's l1: 2.80135	valid_1's l1: 2.79891
[30000]	training's l1: 2.80129	valid_1's l1: 2.7989
[31000]	training's l1: 2.80124	valid_1's l1: 2.7989
[32000]	training's l1: 2.80119	valid_1's l1: 2.7989
[33000]	training's l1: 2.80113	valid_1's l1: 2.7989
[34000]	training's l1: 2.80108	valid_1's l1: 2.7989
[35000]	training's l1: 2.80103	valid_1's l1: 2.7989
[36000]	training's l1: 2.80097	valid_1's l1: 2.7989
[37000]	training's l1: 2.80092	valid_1's l1: 2.7989
[38000]	training's l1: 2.80087	valid_1's l1: 2.7989
Early stopping, best iteration is:
[34344]	training's l1: 2.80106	valid_1's l1: 2.7989
earthquake 9 mae 2.8004529058163463
not seen data mae: 4.9262122734709335

---------------------------------------------------------------------------
KeyboardInterrupt                         Traceback (most recent call last)
<timed eval> in <module>()

<ipython-input-11-cddbb7b7918c> in train_models(earthquake_margin_indexes, complete_earthquakes_length, params, featured_not_seen_data_df, sample_size, holdout_size)
     45                 earthquake_df,
     46                 sample_size=sample_size,
---> 47                 holdout_size=holdout_size
     48             )
     49         X_all = earthquake_add_features_df[earthquake_add_features_df.columns.drop('time_to_failure')]

<ipython-input-3-f3743c33867e> in add_features(df, input_first_index, input_last_index, sample_size, holdout_size, smootch_windows_size)
     77             sample_idx = sample_indexes[i]

In [ ]:
i: 0
complete_earthquakes_length: 43991032
earthquake_margin_indexes: 375377847
Full calculation feature value time (with slicing) 49.16951413551966 min:

/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

Training until validation scores don't improve for 4000 rounds.
[1000]	training's l1: 2.81632	valid_1's l1: 2.81564
[2000]	training's l1: 2.80679	valid_1's l1: 2.80624
[3000]	training's l1: 2.80384	valid_1's l1: 2.80336
[4000]	training's l1: 2.80266	valid_1's l1: 2.80227
[5000]	training's l1: 2.80213	valid_1's l1: 2.80182
[6000]	training's l1: 2.80188	valid_1's l1: 2.80164
[7000]	training's l1: 2.80169	valid_1's l1: 2.80151
[8000]	training's l1: 2.80155	valid_1's l1: 2.80143
[9000]	training's l1: 2.80143	valid_1's l1: 2.80137
[10000]	training's l1: 2.80132	valid_1's l1: 2.80133
[11000]	training's l1: 2.80123	valid_1's l1: 2.80129
[12000]	training's l1: 2.80114	valid_1's l1: 2.80126
[13000]	training's l1: 2.80106	valid_1's l1: 2.80124
[14000]	training's l1: 2.80098	valid_1's l1: 2.80122
[15000]	training's l1: 2.80091	valid_1's l1: 2.8012
[16000]	training's l1: 2.80084	valid_1's l1: 2.80119
[17000]	training's l1: 2.80077	valid_1's l1: 2.80118
[18000]	training's l1: 2.8007	valid_1's l1: 2.80117
[19000]	training's l1: 2.80063	valid_1's l1: 2.80116
[20000]	training's l1: 2.80057	valid_1's l1: 2.80116
[21000]	training's l1: 2.8005	valid_1's l1: 2.80115
[22000]	training's l1: 2.80043	valid_1's l1: 2.80114
[23000]	training's l1: 2.80037	valid_1's l1: 2.80114
[24000]	training's l1: 2.80031	valid_1's l1: 2.80114
[25000]	training's l1: 2.80025	valid_1's l1: 2.80113
[26000]	training's l1: 2.80018	valid_1's l1: 2.80112
[27000]	training's l1: 2.80013	valid_1's l1: 2.80112
[28000]	training's l1: 2.80007	valid_1's l1: 2.80112
[29000]	training's l1: 2.80002	valid_1's l1: 2.80111
[30000]	training's l1: 2.79996	valid_1's l1: 2.80111
[31000]	training's l1: 2.79991	valid_1's l1: 2.80111
[32000]	training's l1: 2.79985	valid_1's l1: 2.80111
[33000]	training's l1: 2.7998	valid_1's l1: 2.80111
Early stopping, best iteration is:
[29588]	training's l1: 2.79999	valid_1's l1: 2.80111
earthquake 0 mae 2.8011210706491614
not seen data mae: 4.927543366133553
i: 1
complete_earthquakes_length: 42442743
earthquake_margin_indexes: 419368879
Full calculation feature value time (with slicing) 48.40985701481501 min:

/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

Training until validation scores don't improve for 4000 rounds.
[1000]	training's l1: 2.7149	valid_1's l1: 2.71412
[2000]	training's l1: 2.70515	valid_1's l1: 2.70452
[3000]	training's l1: 2.70214	valid_1's l1: 2.70158
[4000]	training's l1: 2.70093	valid_1's l1: 2.70044
[5000]	training's l1: 2.70037	valid_1's l1: 2.69994
[6000]	training's l1: 2.70009	valid_1's l1: 2.69973
[7000]	training's l1: 2.69991	valid_1's l1: 2.69961
[8000]	training's l1: 2.69977	valid_1's l1: 2.69953
[9000]	training's l1: 2.69965	valid_1's l1: 2.69948
[10000]	training's l1: 2.69955	valid_1's l1: 2.69943
[11000]	training's l1: 2.69945	valid_1's l1: 2.69939
[12000]	training's l1: 2.69936	valid_1's l1: 2.69936
[13000]	training's l1: 2.69928	valid_1's l1: 2.69933
[14000]	training's l1: 2.69919	valid_1's l1: 2.69931
[15000]	training's l1: 2.6991	valid_1's l1: 2.69927
[16000]	training's l1: 2.69902	valid_1's l1: 2.69924
[17000]	training's l1: 2.69895	valid_1's l1: 2.69922
[18000]	training's l1: 2.69888	valid_1's l1: 2.69921
[19000]	training's l1: 2.69881	valid_1's l1: 2.69919
[20000]	training's l1: 2.69873	valid_1's l1: 2.69918
[21000]	training's l1: 2.69867	valid_1's l1: 2.69917
[22000]	training's l1: 2.6986	valid_1's l1: 2.69916
[23000]	training's l1: 2.69853	valid_1's l1: 2.69915
[24000]	training's l1: 2.69847	valid_1's l1: 2.69915
[25000]	training's l1: 2.6984	valid_1's l1: 2.69914
[26000]	training's l1: 2.69835	valid_1's l1: 2.69914
[27000]	training's l1: 2.69829	valid_1's l1: 2.69914
[28000]	training's l1: 2.69824	valid_1's l1: 2.69913
[29000]	training's l1: 2.69818	valid_1's l1: 2.69913
[30000]	training's l1: 2.69812	valid_1's l1: 2.69913
[31000]	training's l1: 2.69807	valid_1's l1: 2.69913
[32000]	training's l1: 2.69802	valid_1's l1: 2.69912
[33000]	training's l1: 2.69796	valid_1's l1: 2.69912
[34000]	training's l1: 2.69791	valid_1's l1: 2.69913
[35000]	training's l1: 2.69786	valid_1's l1: 2.69912
[36000]	training's l1: 2.6978	valid_1's l1: 2.69913
Early stopping, best iteration is:
[32946]	training's l1: 2.69796	valid_1's l1: 2.69912
earthquake 1 mae 2.6983113676132953
not seen data mae: 4.725996147452754
i: 2
complete_earthquakes_length: 33988602
earthquake_margin_indexes: 461811622
Full calculation feature value time (with slicing) 49.226399354139964 min:

/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

Training until validation scores don't improve for 4000 rounds.
[1000]	training's l1: 2.16846	valid_1's l1: 2.16943
[2000]	training's l1: 2.15907	valid_1's l1: 2.16032
[3000]	training's l1: 2.15609	valid_1's l1: 2.15745
[4000]	training's l1: 2.15489	valid_1's l1: 2.15633
[5000]	training's l1: 2.15433	valid_1's l1: 2.15582
[6000]	training's l1: 2.15405	valid_1's l1: 2.15558
[7000]	training's l1: 2.15388	valid_1's l1: 2.15546
[8000]	training's l1: 2.15376	valid_1's l1: 2.15539
[9000]	training's l1: 2.15366	valid_1's l1: 2.15533
[10000]	training's l1: 2.15357	valid_1's l1: 2.15529
[11000]	training's l1: 2.15348	valid_1's l1: 2.15525
[12000]	training's l1: 2.1534	valid_1's l1: 2.15521
[13000]	training's l1: 2.15332	valid_1's l1: 2.15518
[14000]	training's l1: 2.15325	valid_1's l1: 2.15515
[15000]	training's l1: 2.15317	valid_1's l1: 2.15513
[16000]	training's l1: 2.1531	valid_1's l1: 2.15511
[17000]	training's l1: 2.15303	valid_1's l1: 2.15508
[18000]	training's l1: 2.15297	valid_1's l1: 2.15506
[19000]	training's l1: 2.1529	valid_1's l1: 2.15504
[20000]	training's l1: 2.15284	valid_1's l1: 2.15503
[21000]	training's l1: 2.15277	valid_1's l1: 2.15502
[22000]	training's l1: 2.15271	valid_1's l1: 2.155
[23000]	training's l1: 2.15265	valid_1's l1: 2.15499
[24000]	training's l1: 2.15259	valid_1's l1: 2.15498
[25000]	training's l1: 2.15254	valid_1's l1: 2.15497
[26000]	training's l1: 2.15249	valid_1's l1: 2.15497
[27000]	training's l1: 2.15244	valid_1's l1: 2.15496
[28000]	training's l1: 2.15239	valid_1's l1: 2.15495
[29000]	training's l1: 2.15233	valid_1's l1: 2.15494
[30000]	training's l1: 2.15229	valid_1's l1: 2.15494
[31000]	training's l1: 2.15224	valid_1's l1: 2.15493
[32000]	training's l1: 2.15219	valid_1's l1: 2.15493
[33000]	training's l1: 2.15215	valid_1's l1: 2.15493
[34000]	training's l1: 2.15211	valid_1's l1: 2.15493
[35000]	training's l1: 2.15207	valid_1's l1: 2.15492
[36000]	training's l1: 2.15203	valid_1's l1: 2.15492
[37000]	training's l1: 2.15199	valid_1's l1: 2.15492
[38000]	training's l1: 2.15195	valid_1's l1: 2.15492
[39000]	training's l1: 2.1519	valid_1's l1: 2.15492
[40000]	training's l1: 2.15187	valid_1's l1: 2.15493
Did not meet early stopping. Best iteration is:
[40000]	training's l1: 2.15187	valid_1's l1: 2.15493
earthquake 2 mae 2.1539976661730957
not seen data mae: 3.610067084837362
i: 3
complete_earthquakes_length: 32976890
earthquake_margin_indexes: 495800224
Full calculation feature value time (with slicing) 49.61468665997187 min:

/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

Training until validation scores don't improve for 4000 rounds.
[1000]	training's l1: 2.10847	valid_1's l1: 2.10679
[2000]	training's l1: 2.10037	valid_1's l1: 2.09886
[3000]	training's l1: 2.09785	valid_1's l1: 2.09643
[4000]	training's l1: 2.09684	valid_1's l1: 2.0955
[5000]	training's l1: 2.09636	valid_1's l1: 2.09508
[6000]	training's l1: 2.09615	valid_1's l1: 2.09493
[7000]	training's l1: 2.09602	valid_1's l1: 2.09485
[8000]	training's l1: 2.09592	valid_1's l1: 2.0948
[9000]	training's l1: 2.09582	valid_1's l1: 2.09475
[10000]	training's l1: 2.09574	valid_1's l1: 2.09472
[11000]	training's l1: 2.09566	valid_1's l1: 2.0947
[12000]	training's l1: 2.09559	valid_1's l1: 2.09468
[13000]	training's l1: 2.09552	valid_1's l1: 2.09466
[14000]	training's l1: 2.09546	valid_1's l1: 2.09465
[15000]	training's l1: 2.09539	valid_1's l1: 2.09464
[16000]	training's l1: 2.09534	valid_1's l1: 2.09464
[17000]	training's l1: 2.09528	valid_1's l1: 2.09463
[18000]	training's l1: 2.09522	valid_1's l1: 2.09463
[19000]	training's l1: 2.09517	valid_1's l1: 2.09462
[20000]	training's l1: 2.09511	valid_1's l1: 2.09462
[21000]	training's l1: 2.09506	valid_1's l1: 2.09462
[22000]	training's l1: 2.09501	valid_1's l1: 2.09461
[23000]	training's l1: 2.09496	valid_1's l1: 2.09461
[24000]	training's l1: 2.09491	valid_1's l1: 2.09461
[25000]	training's l1: 2.09486	valid_1's l1: 2.09461
[26000]	training's l1: 2.09482	valid_1's l1: 2.09461
[27000]	training's l1: 2.09477	valid_1's l1: 2.09461
[28000]	training's l1: 2.09473	valid_1's l1: 2.09461
[29000]	training's l1: 2.09468	valid_1's l1: 2.09461
[30000]	training's l1: 2.09464	valid_1's l1: 2.09461
Early stopping, best iteration is:
[26600]	training's l1: 2.09479	valid_1's l1: 2.09461
earthquake 3 mae 2.0972196138207115
not seen data mae: 3.4795987507724666
i: 4
complete_earthquakes_length: 56791029
earthquake_margin_indexes: 528777114
Full calculation feature value time (with slicing) 49.847236017386116 min:

/home/uldo/miniconda3/envs/DS-New/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))

Training until validation scores don't improve for 4000 rounds.
[1000]	training's l1: 3.64726	valid_1's l1: 3.6471
[2000]	training's l1: 3.63791	valid_1's l1: 3.63827
[3000]	training's l1: 3.63505	valid_1's l1: 3.63566
[4000]	training's l1: 3.63383	valid_1's l1: 3.6346
[5000]	training's l1: 3.63325	valid_1's l1: 3.63412
[6000]	training's l1: 3.63296	valid_1's l1: 3.63394
[7000]	training's l1: 3.63276	valid_1's l1: 3.63382
[8000]	training's l1: 3.63258	valid_1's l1: 3.63373
[9000]	training's l1: 3.63243	valid_1's l1: 3.63367
[10000]	training's l1: 3.6323	valid_1's l1: 3.63363
[11000]	training's l1: 3.63217	valid_1's l1: 3.63359
[12000]	training's l1: 3.63206	valid_1's l1: 3.63355
[13000]	training's l1: 3.63195	valid_1's l1: 3.63353
[14000]	training's l1: 3.63186	valid_1's l1: 3.63351
[15000]	training's l1: 3.63176	valid_1's l1: 3.63349
[16000]	training's l1: 3.63167	valid_1's l1: 3.63348
[17000]	training's l1: 3.63158	valid_1's l1: 3.63346
[18000]	training's l1: 3.63148	valid_1's l1: 3.63345
[19000]	training's l1: 3.6314	valid_1's l1: 3.63344
[20000]	training's l1: 3.63131	valid_1's l1: 3.63342
[21000]	training's l1: 3.63123	valid_1's l1: 3.63341
[22000]	training's l1: 3.63115	valid_1's l1: 3.6334
[23000]	training's l1: 3.63108	valid_1's l1: 3.63339
[24000]	training's l1: 3.63101	valid_1's l1: 3.63339
[25000]	training's l1: 3.63093	valid_1's l1: 3.63339
[26000]	training's l1: 3.63086	valid_1's l1: 3.63339
[27000]	training's l1: 3.63079	valid_1's l1: 3.63339
[28000]	training's l1: 3.63072	valid_1's l1: 3.63339
Early stopping, best iteration is:
[24656]	training's l1: 3.63096	valid_1's l1: 3.63339
earthquake 4 mae 3.6311940851279476
not seen data mae: 6.62556275849406
CPU times: user 1d 2h 40min 44s, sys: 2min 38s, total: 1d 2h 43min 22s
Wall time: 6h 41min 24s
